In [16]:
import numpy as np
import cv2
import random
import os

In [24]:
path = r"E:\Users\Morgan\Doccuments\UniMLProject\raw_datasets\WIDER_val\wider_face_val_bbx_gt.txt"
path_to_wider_imgs = r"E:\Users\Morgan\Doccuments\UniMLProject\raw_datasets\WIDER_val\images"

In [13]:
def get_training_data(path):
    '''Returns two lists: a list of the names of the image files and a list of the bounding boxes'''
    #Open file
    f = open(path,'r') 
    file = f.readlines()
    name = []
    box = []
    count = 0
    for i in file: 
        #Grab the files with only 1 face
        if i == '1\n':
            filename = file[count-1]
            bb = file[count+1]
        
            #Get rid of the /n
            filename = filename[:-1]
            #Convert into array of integer coordinates
            num = bb.split(' ')
            num = num[:4]
            num = [int(i) for i in num]
            
            #Save into array
            name.append(filename)
            box.append(num)
        count = count+1
    return name, box

In [3]:
def delete_zeros(name, box):
    '''Gets rid of errors in which bounding box coordinates are 0'''
    error=[]
    c=0
    for b in box:
        if b==[0,0,0,0]:
            error.append(c)
        c=c+1
    for e in error:
        del box[e]
        del name[e]
    return name, box

In [4]:
def save_coordinates(coordinates):
    '''Saves bounding box coordinates in txt file'''
    F = open('./coordinates.txt','w')
    for line in coordinates:
        line=line+'\n'
        F.write(line)

In [5]:
def create_scales(box):
    '''Create a list of scales that result in the face being 7,8,9,10,11,and 12 pixels big'''
    scale = np.empty([6])
    facesize = [12,11,10,9,8,7]
    width = box[2]
    height = box[3]
    larger = max(height, width)
    scale = [larger/i for i in facesize]
    return scale

In [6]:
def scale_image(image,scale):
    '''Scales image given the scale'''
    height, width, _ = image.shape
    width_scaled = int(np.ceil(width / scale))
    height_scaled = int(np.ceil(height / scale))
    im_data = cv2.resize(image, (width_scaled, height_scaled), interpolation=cv2.INTER_AREA)
    return im_data

In [7]:
def scale_box(box,scale):
    '''Scales bounding box given the scale'''
    box = [int(i/scale) for i in box]
    return box

In [36]:
def gen_pos_training(name, bbox, count):
    '''Generates 12x12 pixel images with faces. This probably could have been written more concisely...but oh well, here it is'''
    image = cv2.imread("./raw_datasets/WIDER_val/images/"+name)
    scales=create_scales(bbox)
    print(scales)
    bbox = [bbox[0],bbox[1],bbox[0]+bbox[2],bbox[1]+bbox[3]]
    coordinates = []
    
    #Scale to 12
    img=scale_image(image,scales[0])
    box = scale_box(bbox,scales[0])
    #If height > width
    if box[3]-box[1] >= box[2]-box[0]:
        x=box[2]-12
        y=box[3]-12
        while x<=box[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
    #If width > height
    elif bbox[3]-bbox[1] <= bbox[2]-bbox[0]:
        x=bbox[2]-12
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
    #If equal
    elif bbox[3]-bbox[1] == bbox[2]-bbox[0]:
        crop_img = img[bbox[1]:bbox[3],box[1]:bbox[2]]
        cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
        coordinates.append(f'./pos_train/{count}.bmp: [0,0,1,1]')
        count = count+1
    
    #Scale to 11
    img=scale_image(image,scales[1])
    box = scale_box(bbox,scales[1])
    #If height > width
    if box[3]-box[1] > box[2]-box[0]:
        x=box[2]-12
        y=box[3]-12
        while x<=box[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
        x=box[2]-12
        y=box[3]-11
        while x<=box[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
    #If width > height
    elif bbox[3]-bbox[1] < bbox[2]-bbox[0]:
        x=bbox[2]-12
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
        x=bbox[2]-11
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
    #If equal
    elif bbox[3]-bbox[1] == bbox[2]-bbox[0]:
        x=bbox[2]-12
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
        x=bbox[2]-12
        y=bbox[3]-12
        while x<=bbox[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
    
    #Scale to 10
    img=scale_image(image,scales[2])
    box = scale_box(bbox,scales[2])
    #If height > width
    if box[3]-box[1] > box[2]-box[0]:
        x=box[2]-12
        y=box[3]-12
        while x<=box[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
        x=box[2]-12
        y=box[3]-11
        while x<=box[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
        x=box[2]-12
        y=box[3]-10
        while x<=box[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
    #If width > height
    elif bbox[3]-bbox[1] < bbox[2]-bbox[0]:
        x=bbox[2]-12
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
        x=bbox[2]-11
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
        x=bbox[2]-10
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
    #If equal
    elif bbox[3]-bbox[1] == bbox[2]-bbox[0]:
        x=bbox[2]-12
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
        x=bbox[2]-12
        y=bbox[3]-12
        while x<=bbox[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
            
    #Scale to 9
    img=scale_image(image,scales[3])
    box = scale_box(bbox,scales[3])
    #If height > width
    if box[3]-box[1] > box[2]-box[0]:
        x=box[2]-12
        y=box[3]-12
        while x<=box[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
        x=box[2]-12
        y=box[3]-11
        while x<=box[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
        x=box[2]-12
        y=box[3]-10
        while x<=box[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
        x=box[2]-12
        y=box[3]-9
        while x<=box[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
    #If width > height
    elif bbox[3]-bbox[1] < bbox[2]-bbox[0]:
        x=bbox[2]-12
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
        x=bbox[2]-11
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
        x=bbox[2]-10
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
        x=bbox[2]-9
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
    #If equal
    elif bbox[3]-bbox[1] == bbox[2]-bbox[0]:
        x=bbox[2]-12
        y=bbox[3]-12
        while y<=bbox[1]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            y = y+1
        x=bbox[2]-12
        y=bbox[3]-12
        while x<=bbox[0]:
            crop_img = img[y:y+12,x:x+12]
            if crop_img.shape == (12,12,3):
                cv2.imwrite(f'./pos_train/{count}.bmp',crop_img)
                newboxA=[box[0]-x,box[1]-y,box[2]-x,box[3]-y]
                newboxB=[i/12 for i in newboxA]
                coordinates.append(f'{count}.bmp: [{newboxB[0]},{newboxB[1]},{newboxB[2]},{newboxB[3]}]')
                count = count+1
            x = x+1
    return coordinates, count

In [37]:
def gen_neg_training(name, bbox, count):
    '''Generates 12x12 pixel images with no faces'''
    path =os.path.join("./raw_datasets/WIDER_val/images/",name)
    print(path)
    image = cv2.imread(path)
    scales=create_scales(bbox)
    print(scales)
    bbox = [bbox[0],bbox[1],bbox[0]+bbox[2],bbox[1]+bbox[3]]
    a = name.split('/')
    b = a[1][:-4]
    
    #For each scale:
    for j in range(4):
        img=scale_image(image,scales[j])
        box = scale_box(bbox,scales[j])
        height, width, _ = img.shape
        i=0
        y1=0
        if width<=13 or height<=13:
            i=16
        while i<15:
            x=random.randint(0,width-13)
            y=random.randint(0,height-13)
            if y1>100: #If something went wrong and it keeps on falling into "continue", just break out of the loop
                i=16
                break
            if [x,y]>[box[0]-12,box[1]-12] and [x,y]<[box[2],box[3]]: #If the 12x12 box overlaps with the bounding box, draw another box
                y1=y1+1
                continue
            else: #If it doesn't overlap, crop the image.
                crop_img = img[y:y+12,x:x+12]
                cv2.imwrite(f'./neg_train/{count}.bmp',crop_img)
                count = count+1
                i=i+1
    print("count",count)
    return count

In [38]:
name,box = get_training_data(path)
name,box = delete_zeros(name,box)

In [39]:
pcount=1
ncount=1
coordinateA=[]
for i in range(100):
    print(i)
    ncount = gen_neg_training(name[i],box[i],ncount)
    coordinateB, pcount = gen_pos_training(name[i],box[i],pcount)
    coordinateA=coordinateA+coordinateB
    print('ncount:',ncount)
    print('pcount:',pcount)
save_coordinates(coordinateA)

0
./raw_datasets/WIDER_val/images/0--Parade/0_Parade_Parade_0_829.jpg
[36.916666666666664, 40.27272727272727, 44.3, 49.22222222222222, 55.375, 63.285714285714285]
count 3
[36.916666666666664, 40.27272727272727, 44.3, 49.22222222222222, 55.375, 63.285714285714285]
ncount: 3
pcount: 67
1
./raw_datasets/WIDER_val/images/0--Parade/0_Parade_marchingband_1_818.jpg
[46.083333333333336, 50.27272727272727, 55.3, 61.44444444444444, 69.125, 79.0]
count 18
[46.083333333333336, 50.27272727272727, 55.3, 61.44444444444444, 69.125, 79.0]
ncount: 18
pcount: 98
2
./raw_datasets/WIDER_val/images/0--Parade/0_Parade_Parade_0_246.jpg
[49.166666666666664, 53.63636363636363, 59.0, 65.55555555555556, 73.75, 84.28571428571429]
count 18
[49.166666666666664, 53.63636363636363, 59.0, 65.55555555555556, 73.75, 84.28571428571429]
ncount: 18
pcount: 98
3
./raw_datasets/WIDER_val/images/0--Parade/0_Parade_marchingband_1_356.jpg
[9.5, 10.363636363636363, 11.4, 12.666666666666666, 14.25, 16.285714285714285]
count 78
[9.

[28.25, 30.818181818181817, 33.9, 37.666666666666664, 42.375, 48.42857142857143]
count 1094
[28.25, 30.818181818181817, 33.9, 37.666666666666664, 42.375, 48.42857142857143]
ncount: 1094
pcount: 1284
29
./raw_datasets/WIDER_val/images/11--Meeting/11_Meeting_Meeting_11_Meeting_Meeting_11_865.jpg
[35.166666666666664, 38.36363636363637, 42.2, 46.888888888888886, 52.75, 60.285714285714285]
count 1094
[35.166666666666664, 38.36363636363637, 42.2, 46.888888888888886, 52.75, 60.285714285714285]
ncount: 1094
pcount: 1337
30
./raw_datasets/WIDER_val/images/11--Meeting/11_Meeting_Meeting_11_Meeting_Meeting_11_320.jpg
[19.166666666666668, 20.90909090909091, 23.0, 25.555555555555557, 28.75, 32.857142857142854]


KeyboardInterrupt: 